In [1]:
import torch

if torch.cuda.is_available():
    print("GPU is available")
else:
    print("GPU is not available")

GPU is available


In [3]:
!pip install -q langchain langchain-community pandas faiss-cpu tiktoken transformers chromadb sentence-transformers gradio

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 6.1 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.4/50.4 kB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 50.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 81.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 27.0/27.0 MB 73.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 58.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 584.3/584.3 kB 40.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 76.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 227.1/227.1 kB 20.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.8/16.8 MB 66.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 318.7/318.7 kB 23.7 MB/s eta 0:00

In [7]:
!mkdir -p data
!wget "https://www.dropbox.com/s/948jr9cfs7fgj99/UBER.zip?dl=1" -O data/UBER.zip
!unzip -q data/UBER.zip -d data

# Confirm the files are extracted
!ls data/UBER/

--2024-09-02 09:49:51--  https://www.dropbox.com/s/948jr9cfs7fgj99/UBER.zip?dl=1
Resolving www.dropbox.com (www.dropbox.com)... 162.125.81.18, 2620:100:601d:18::a27d:512
Connecting to www.dropbox.com (www.dropbox.com)|162.125.81.18|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://www.dropbox.com/scl/fi/vetj6tgcux8e309swquxs/UBER.zip?rlkey=oy4vx60rplorounkh9wh2waux&dl=1 [following]
--2024-09-02 09:49:52--  https://www.dropbox.com/scl/fi/vetj6tgcux8e309swquxs/UBER.zip?rlkey=oy4vx60rplorounkh9wh2waux&dl=1
Reusing existing connection to www.dropbox.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://uc2b527e2772c31b8e307b1cc5fb.dl.dropboxusercontent.com/cd/0/inline/CZz_0Bj-E0MOND0Z2cOH65YZleLpZLVtpYVxaK0xqUea0dZQclcbG3cKGPUZ99gRG_mfju9_4J2oiQ4QNgRALNbXuP8ED3zMR1pbeJEShGfkV6mZ0IrxUUcsbsx9PPo70xs/file?dl=1# [following]
--2024-09-02 09:49:53--  https://uc2b527e2772c31b8e307b1cc5fb.dl.dropboxusercontent.com/cd/0/inline/CZz_0Bj-E0MO

In [ ]:
import os
import pandas as pd
from lxml import etree

def parse_xbrl(file_path):
    """
    Parse an XBRL (eXtensible Business Reporting Language) file and extract relevant data into a DataFrame.

    Args:
        file_path (str): The path to the XBRL file to be parsed.

    Returns:
        pd.DataFrame: A DataFrame containing the extracted data with 'Element' and 'Value' columns.
                      Returns an empty DataFrame if an error occurs.
    """
    try:
        tree = etree.parse(file_path)
        root = tree.getroot()

        # Define the XBRL namespace (adjust if needed)
        namespaces = {'xbrl': 'http://www.xbrl.org/2003/instance'}

        # Extract all relevant data (adjust XPath as needed)
        data = []
        for element in root.findall('.//xbrl:*', namespaces=namespaces):
            if element.text and element.text.strip():
                data.append({
                    'Element': element.tag,
                    'Value': element.text.strip()
                })

        # Convert list to DataFrame
        df = pd.DataFrame(data)

        return df

    except Exception as e:
        print(f"Error processing {file_path}: {e}")
        return pd.DataFrame()  # Return an empty DataFrame on error

# Directory containing XBRL HTML files
data_dir = "data/UBER"

# List all XBRL HTML files in the directory
files = os.listdir(data_dir)
xbrl_files = [f for f in files if f.endswith('.html')]

# Process each file and collect DataFrames
dfs = []
for file in xbrl_files:
    file_path = os.path.join(data_dir, file)
    df = parse_xbrl(file_path)
    if not df.empty:
        dfs.append(df)

# Combine all DataFrames into one DataFrame
if dfs:
    combined_df = pd.concat(dfs, ignore_index=True)
    print("Combined DataFrame:")
    print(combined_df.head())
else:
    print("No data was processed.")


In [15]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

documents = combined_df['Value'].dropna().astype(str).tolist()

vectorizer = TfidfVectorizer()
X = vectorizer.fit_transform(documents)

import joblib
joblib.dump(vectorizer, 'vectorizer.pkl')
joblib.dump(documents, 'documents.pkl')


['documents.pkl']

In [16]:
import faiss
import numpy as np

dimension = X.shape[1]
index = faiss.IndexFlatL2(dimension)
index.add(X.toarray())

faiss.write_index(index, 'faiss_index.index')


In [17]:
from langchain.chains import RetrievalQA
from langchain.llms import HuggingFacePipeline
from transformers import pipeline
import faiss
import joblib

vectorizer = joblib.load('vectorizer.pkl')
documents = joblib.load('documents.pkl')
index = faiss.read_index('faiss_index.index')

generator = pipeline("text-generation", model="distilgpt2")


config.json:   0%|          | 0.00/762 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/353M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

Hardware accelerator e.g. GPU is available in the environment, but no `device` argument is passed to the `Pipeline` object. Model will be on CPU.
Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Context: 2020-07-01

Question: What is the entity information?
Question: What is the individual information contained in the content?
Question: What are the components of the entity information contained within the content?
Question: What


In [ ]:
def query_vector_store(query):
    """
    Transform a query into a vector and retrieve the most similar document from the vector store.

    Args:
        query (str): The query string to be transformed and used for retrieval.

    Returns:
        str: The most similar document from the documents list based on the query vector.
    """
    query_vector = vectorizer.transform([query]).toarray()
    _, indices = index.search(query_vector, k=1)
    return documents[indices[0][0]]

def chatbot_response(question):
    """
    Generate a response to a question using the context of the most similar document retrieved from the vector store.

    Args:
        question (str): The question string to generate a response for.

    Returns:
        str: The generated response based on the retrieved document and question.
    """
    retrieved_document = query_vector_store(question)
    context = f"Context: {retrieved_document}\n\nQuestion: {question}"
    response = generator(context, max_length=50, num_return_sequences=1)
    return response[0]['generated_text']

print(chatbot_response("What is the entity information?"))



In [18]:
import gradio as gr

def gradio_chatbot_response(question):
    return chatbot_response(question)

interface = gr.Interface(fn=gradio_chatbot_response, inputs="text", outputs="text")
interface.launch()



Setting queue=True in a Colab notebook requires sharing enabled. Setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
Running on public URL: https://e611371d7381ebbc82.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)
